In [1]:
class LaTeXElement:
    def __init__(self):
        self.attributes = {}
    
    def render(self):
        raise NotImplementedError("Subclasses should implement this method.")

class TextSection(LaTeXElement):
    def __init__(self, text):
        super().__init__()
        self.text = text

    def render(self):
        # Render the text section in LaTeX
        raise NotImplementedError("Subclasses should implement this method.")

In [22]:
class Table(LaTeXElement):
    def __init__(self, num_rows, num_cols, headers=None, border=True, row_shading = False):
        super().__init__()
        self.num_rows = num_rows
        self.num_cols = num_cols
        self.headers = headers
        self.border = border
        self.row_shading = row_shading
        self.cells = [["" for _ in range(num_cols)] for _ in range(num_rows)]

    def set_cell(self, row, col, content):
        self.cells[row][col] = content

    def render(self):
        # Start the tabular environment based on whether the table has borders
        tabular_line = " | ".join(["l" for _ in range(self.num_cols)])
        tabular_line = f"\\begin{{tabular}}{{|{tabular_line}|}}" if self.border else f"\\begin{{tabular}}{{{tabular_line}}}"
        table_code = [tabular_line, "\\hline"] if self.border else [tabular_line]

        # Add the headers if they exist
        if self.headers:
            table_code.append(" & ".join(self.headers) + " \\\\\\hline" if self.border else " & ".join(self.headers) + " \\\\")

        # Add the content of the cells
        for row_index, row in enumerate(self.cells):
            if self.row_shading and row_index % 2 == 1:
                table_code.append(r"\rowcolor{gray!20}")
            row_content = " & ".join(row)
            table_code.append(row_content + " \\\\\\hline" if self.border else row_content + " \\\\")
        
        # Close the tabular environment
        table_code.append("\\end{tabular}")

        return "\n".join(table_code)


class UniversityTitle(TextSection):
    def __init__(self, title):
        super().__init__(title)

    def render(self):
        return f"\\begin{{center}}\n\\textbf{{\\Large {self.text}}}\n\\end{{center}}\n\\vspace{{2mm}}\n"
    
class PersonalInformation(LaTeXElement):
    def __init__(self, name, student_id):
        super().__init__()
        self.name = name
        self.student_id = student_id

    def render(self):
        return (
            f"Name: \\textbf{{ {self.name} }}\\\\\n"
            f"Student ID: \\textbf{{ {self.student_id} }}\n\\vspace{{4mm}}\n"
        )

In [38]:
class ImageSection(LaTeXElement):
    def __init__(self, path, is_watermark=False):
        super().__init__()
        self.path = path
        self.is_watermark = is_watermark

    def render(self):
        # Render the image in LaTeX
        if self.is_watermark:
            # If it's a watermark, return LaTeX command to put image in the background
            return (
                r"\AddToShipoutPicture*{" "\n"
                r"\put(0,0){" "\n"
                r"\includegraphics[width=\paperwidth,height=\paperheight]{" + self.path + "}}" "\n"
                r"}"
            )
        else:
            # Regular image placement
            return r"\includegraphics{" + self.path + "}"


In [50]:
class Document:
    def __init__(self, double_column=False, paper_size='a4paper', use_fancy_header=False, page_count=False,
                 font_type='times', header_layout = None):
        
        self.elements = []
        self.double_column = double_column
        self.paper_size = paper_size
        self.use_fancy_header = use_fancy_header
        self.page_count = page_count
        self.font_type = font_type
        self.header_layout = header_layout

    def add_element(self, element):
        if isinstance(element, LaTeXElement):
            self.elements.append(element)
        else:
            raise TypeError("element must be an instance of LaTeXElement")

    def render_preamble(self):
        preamble = [
            r"\documentclass[12pt, " + self.paper_size + r"]{article}",
            r"\usepackage[utf8]{inputenc}",
            r"\usepackage{geometry}",
            r"\geometry{margin=1in}",  # Set the margins
            r"\usepackage{eso-pic}", # For Watermarks
            r"\usepackage{multicol}",
            r"\usepackage{fancyhdr}",
            r"\usepackage{" + self.font_type + "}" if self.font_type else "",
            r"\renewcommand{\familydefault}{\sfdefault}",
        ]
        if self.use_fancy_header:
            preamble.extend([
                r"\pagestyle{fancy}",
                r"\fancyhf{}",
            ])

        if self.header_layout is not None:
            preamble.extend([
                self.header_layout
            ])
        # Begin document
        preamble.append(r"\begin{document}")
        
        if self.use_fancy_header:
            preamble.append(r"\thispagestyle{fancy}")  # Use fancy header and footer on this page
        return "\n".join(preamble)

    def render(self):
        # Render the entire document in LaTeX
        document = [self.render_preamble()]
            
        if self.double_column:
            document.append(r"\begin{multicols}{2}")
        for element in self.elements:
            document.append(element.render())
        if self.double_column:
            document.append(r"\end{multicols}")
        document.append(r"\end{document}")

        return '\n'.join(document)

In [51]:
# Create the document object
transcript_document = Document(double_column=True, paper_size='a4paper', use_fancy_header=True, page_count=True, 
                               font_type='helvet')
# Add University Title
uni_title = UniversityTitle("UNOFFICIAL University at Buffalo Transcript")
transcript_document.add_element(uni_title)

# Add Personal Information Section
personal_info = PersonalInformation(name="Jeremy Robert Chapman", student_id="501-9627")
transcript_document.add_element(personal_info)

# Create a table for courses
course_table = Table(num_rows=5, num_cols=4, headers=["Course", "Description", "Attempted", "Grade"])
# Populate the table with sample data
sample_courses = [
    {"code": "MTH101", "description": "Calculus I", "attempted": "4.0", "grade": "A"},
    {"code": "PHY101", "description": "Physics I", "attempted": "4.0", "grade": "B"},
    # ... more sample courses ...
]
for row, course in enumerate(sample_courses):
    course_table.set_cell(row, 0, course["code"])
    course_table.set_cell(row, 1, course["description"])
    course_table.set_cell(row, 2, course["attempted"])
    course_table.set_cell(row, 3, course["grade"])
transcript_document.add_element(course_table)

# Add watermark
watermark_image = ImageSection('transcript_generator/queens_logo.png', is_watermark=True)
transcript_document.add_element(watermark_image)
# Render the document to LaTeX code
latex_code = transcript_document.render()

# The `latex_code` variable now holds the LaTeX code for the subset of the transcript.

In [52]:
print(latex_code)

\documentclass[12pt, a4paper]{article}
\usepackage[utf8]{inputenc}
\usepackage{geometry}
\geometry{margin=1in}
\usepackage{eso-pic}
\usepackage{multicol}
\usepackage{fancyhdr}
\usepackage{helvet}
\renewcommand{\familydefault}{\sfdefault}
\pagestyle{fancy}
\fancyhf{}
\begin{document}
\thispagestyle{fancy}
\begin{multicols}{2}
\begin{center}
\textbf{\Large UNOFFICIAL University at Buffalo Transcript}
\end{center}
\vspace{2mm}

Name: \textbf{ Jeremy Robert Chapman }\\
Student ID: \textbf{ 501-9627 }
\vspace{4mm}

\begin{tabular}{|l | l | l | l|}
\hline
Course & Description & Attempted & Grade \\\hline
MTH101 & Calculus I & 4.0 & A \\\hline
PHY101 & Physics I & 4.0 & B \\\hline
 &  &  &  \\\hline
 &  &  &  \\\hline
 &  &  &  \\\hline
\end{tabular}
\AddToShipoutPicture*{
\put(0,0){
\includegraphics[width=\paperwidth,height=\paperheight]{transcript_generator/queens_logo.png}}
}
\end{multicols}
\end{document}


In [54]:
import os
import subprocess

# Write the LaTeX code to a file
with open('document.tex', 'w') as file:
    file.write(latex_code)

# Compile the .tex file to PDF with pdflatex
# Make sure the pdflatex command is available on your system path
result = subprocess.run(
    ['pdflatex', '-interaction=nonstopmode', 'document.tex'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Check if pdflatex command was successful
if result.returncode != 0:
    # Handle the error case
    print("pdflatex command failed.")
    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)
else:
    print("PDF was successfully created.")

# Clean up auxiliary files generated by pdflatex (optional)
for ext in ('.aux', '.log', '.out'):
    try:
        os.remove(f'document{ext}')
    except OSError:
        pass

pdflatex command failed.
STDOUT: This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./document.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-02-22>
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/article.cls
Document Class: article 2022/07/02 v1.4n Standard LaTeX document class
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/size12.clo))
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/local/texlive/2023/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/local/texlive/2023/texmf-dist/tex/generic/iftex/ifvtex.sty
(/usr/local/texlive/2023/texmf-dist/tex/generic/iftex/iftex.sty)))
(/usr/local/texlive/2023/texmf-dist/tex/latex/eso-pic/eso-pic.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/xcolor/xcolor.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/graphics-cfg/color.cfg)
(/usr/l